# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [9]:
install.packages("httr")
install.packages("tokenizers")
library(httr)
library(tokenizers)

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}

Warning message:
"package 'httr' is in use and will not be installed"
Installing package into 'C:/Users/keert/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependency 'SnowballC'




package 'SnowballC' successfully unpacked and MD5 sums checked
package 'tokenizers' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\keert\AppData\Local\Temp\RtmpKg3KNH\downloaded_packages


Warning message:
"package 'tokenizers' was built under R version 4.4.3"


#### b) Make a function generate keys for ngrams.

In [10]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}

#### c) Make a function to build an ngram table.

In [11]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))
    tbl <- new.env(parent = emptyenv())
    for (i in seq_len(length(tokens) - n + 1L)) {
        ngram <- tokens[i:(i + n - 2L)]
        next_word <- tokens[i + n - 1L]
        key <- paste(ngram, collapse = sep)
        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }
    tbl
}

#### d) Function to digest the text.

In [12]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [13]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt,n)
}

#### f) Function that gives random start.

In [14]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names=TRUE)
    if (length(keys)==0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [15]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [16]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [17]:
# a) i) Using it (n=3) start word "the king"
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"

set.seed(2025)
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

# Generate text starting with "the king"
gen3(start_words = c("the", "king"), length = 15)

[1] "the king has forbidden me to marry another husband am not i shall ride upon"

In [18]:
# a) ii) 
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"

set.seed(2025)
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

# Generate random start
gen3(length = 15)

[1] "spread the jam over it spread its wings and crying here comes our hobblety jib"

#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [21]:
# b) i) Using it (n=3) start word "the king"
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"

set.seed(2025)
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

# Generate text starting with "the king"
gen3(start_words = c("the", "king"), length = 15)

[1] "the king he added to the entire exclusion of the swords were made prisoners the"

In [22]:
# b) ii) 
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"

set.seed(2025)
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

# Generate random start
gen3(length = 15)

[1] "laces of silk d milōn le cuireu͂r xxxviij pa͂r alec͂t co͂r p'̄c pa͂r iiij d"

#### c) Explain in 1-2 sentences the difference in content generated from each source.

The content generated from each source is different because the vocabulary and the ngrams are different between the sources. Because sampling is probabalistic, how often a word appears also differs and the starting word changes based on whether you set one or if it is random.

## Question 3
#### a) What is a language learning model? 
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

a) A language learning model is a type of machine learning model that predicts the probability of a sequence of words to understand and generate human language

b) You can run a language model locally by sending HTTP API requests through a program

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | a program that interprets the command. The shell parses the command and launches the mkdir process with "project" as an argument |
| **Terminal emulator** | hosts the shell. shows the text you type and the output you get so in this case the "mkdir project' |
| **Process** | something running on your computer. The shell creates a new process that runs the mkdir program. |
| **Signal** | something that you can can send to processes to tell them to do something. This example doesn't normally use a signal but you can send a signal to interrupt it |
| **Standard input** | something that each process has - A text stream that a process can read from |
| **Standard output** | something that each process has - A text stream that a process writes normal output to |
| **Command line argument** | the things that you can pass to a process when you start it - the word "project" is the command-line argument telling mkdir which directory to create|
| **The environment** | all the stuff a process can see when its running - in this case a set of variables passed to the mkdir process by the shell |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

a) the programs are:
find – searches for files and directories \
xargs – builds command lines from input \
grep – searches text inside files 

b) This command is finding searches starting in the current directory (.) \
-iname "*.R" finds files whose names end in .R. \
Output is a list of .R file paths, one per line. \
the pipe takes the output of find and uses it as the input to xargs \
xargs takes all the filenames from find and appends them to the end of a grep command \
grep read_csv searches each file for lines that contain the text read_csv

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

a) Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/


b) docker run -d -p 8787:8787 -e PASSWORD=docker! --name rstudio rocker/rstudio      4dc039b79aa0aae501bc831a5e519ea58f1474257ecef9b24922b3b8e3267771

I logged into the Rstudio using my server and password